# Various Things

In [2]:
import rioxarray as rxr
import rasterio as rio
import pandas as pd
import geopandas as gpd

## Testing on S2 image

For Ana

In [5]:
img = rxr.open_rasterio("/Users/lestes/images/s2/sudan_example_lateWA.tif")

Subset and write to COGs

In [8]:
img_a = img.isel(band=[0,1,2,3])
img_b = img.isel(band=[4,5,6,7])

In [13]:
img_a.rio.to_raster("/Users/lestes/images/s2/sudan_example_lateWA_a.tif", 
                    cog = True)
img_b.rio.to_raster("/Users/lestes/images/s2/sudan_example_lateWA_b.tif", 
                    cog = True)


Make a catalog

In [19]:
# catalog = gpd.read_file("../data/mappingafrica-tile-catalog-small.geojson")
catalog = pd.DataFrame({
    "name": ["sudan_example_lateWA_a", "sudan_example_lateWA_b"],
    "date": ["2024-05-01", "2024-10-01"],
    "window_b": ["sudan_example_lateWA_a.tif", 
                 "sudan_example_lateWA_b.tif"],
})
catalog.to_csv("../data/sudan-test-catalog.csv", index=False)


#### Catalog for Kenya Planet

In [ ]:
aoi = gpd.read_file("../external/aois/lindquist-aoi.kml")

pth = "~/Dropbox/projects/activelearning/mappingafrica/campaigns/data/grids"
tiles = gpd.read_file(f"{pth}/kenya_tiles.geojson")[["tile", "geometry"]]

(tiles.sjoin(aoi, how="inner", predicate="intersects")[["tile", "geometry"]]
 .reset_index(drop=True)
 .to_file("../data/kenya-lindqvist-tiles.geojson", driver="GeoJSON"))


# tiles = [gpd.read_file(f"{pth}/{cntry}_tiles.geojson")[["tile", "geometry"]] 
#          for cntry in ["kenya"]]
# # tiles = pd.concat(tiles, axis=0)
# tiles["tile"] = tiles["tile"].astype(int).astype(str)
# tiles = tiles.query("tile in @tile_ids").reset_index(drop=True)

# # tiles_gdf = (
# #     pd.merge(tile_catalog, tiles, left_on="name", right_on="tile", how="inner")
# #     .drop(columns=["tile"])
# #     .pipe(gpd.GeoDataFrame, geometry="geometry", crs="EPSG:4326")
# # )
# # tiles_gdf.to_file("../data/mappingafrica-tile-catalog-small.geojson", 
# #                   driver="GeoJSON")


Tanmoy made catalogs for running our original model. Combine these with buffered tiles I made in mappingafrica repo

In [26]:
# catalogs
kenya_cats = []
for i in range(2018, 2025):
    kenya_cats.append(
        pd.read_csv(f"../external/catalogs/prediction_catalog_kenya_{i}.csv")
    )

kenya_cats = pd.concat(kenya_cats, axis=0).reset_index(drop=True)
kenya_cats["date"] = kenya_cats["year"].apply(lambda x: f"{x}-07-15")
kenya_cats = kenya_cats.rename(
    columns={"tile": "name", "dir_img": "window_b"}
)[["name", "date", "window_b"]]

buf_tiles = gpd.read_file(
    f"~/Dropbox/projects/activelearning/mappingafrica/campaigns/data/grids/"
    "retile/kenya_tiles_buf184.geojson"
)

tiles_gdf = pd.merge(
    kenya_cats[["name", "date", "window_b"]], 
    buf_tiles[["tile", "geometry"]], 
    how="inner", 
    left_on="name", 
    right_on="tile"
)
tiles_gdf = gpd.GeoDataFrame(
    tiles_gdf.drop(columns=["tile"]),
    geometry="geometry", 
    crs="EPSG:4326"
)

tiles_gdf.to_file(
    f"../external/catalogs/mappingafrica-kenya-buffered-tile-catalog.geojson", 
    driver="GeoJSON"
)